Dash samples  
https://github.com/plotly/dash-sample-apps/tree/main

https://proglib.io/p/tutorial-vizualizaciya-dannyh-v-vebe-s-pomoshchyu-python-i-dash-2021-01-11

https://readmedium.com/dashboards-in-python-for-beginners-and-everyone-else-using-dash-f0a045a86644

pip install dash==1.0.2  # The core dash backend <br>
pip install dash-daq==0.1.0  # DAQ components (newly open-sourced!)

Начиная с какой то версии dash автоматически инсталлит базовые зависимости, больше нет нужды перечилять их все

In [ ]:
!pip install dash

In [ ]:
!pip install dash-daq

In [1]:
from dash import dcc
print(dcc.__version__)

2.13.0


In [ ]:
!pip install pyngrok

In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input

dash app состоит из 2 компонент
 - layout - расположение элементов, древовидная структура отображаемых элементов, в общем все то что мы видим
 - callback - реактовая компонента, отвечает за взаимодейсвтие

To run the app, copy the above code into a new file named app.py and type into your terminal the command python app.py.   
Then, go to the http link.

The children property is special. By convention, it's always the first attribute which means that you can omit it: html.H1(children='Hello Dash')   
is the same as html.H1('Hello Dash'). Also, it can contain a string, a number, a single component, or a list of components.    
Some components, such as html.Div and html.P, accept a value for their children prop. Others,    
such as dcc.Graph or dcc.Dropdown do not,  and require other props in order to function properly.  
As @KarlKnechtel mentioned in his comment, when one component is a child of another,     
it represents a nesting of the first component inside the other. The following are analogous:  

In [ ]:
html.Div(
    children=[
        html.H1('This is some text'),
        html.P('This is also some text'),
    ]
)

Чтобы отключить у графико плавающую панель инструментов Plotyly (modebar)  

In [ ]:
children=dcc.Graph(
                id="price-chart",
                config={"displayModeBar": False}
                ...)

style

In [ ]:
html.Div(children=[],
        style= {'width': '49%'
                , 'display': 'inline-block'
                , 'padding': 10
                , 'padding': '10px 5px'
                , 'padding': '0px 20px 20px 20px'
                , 'flex': 1
                , 'textAlign': 'center'
                , 'color': 'blue'
                , 'display': 'flex'
                , 'flexDirection': 'row'
                , 'width': '48%'
                , 'float': 'right'
                , 'fontSize': '16px'
                , "justifyContent":"center"                
        }
)        

Dcc и расположение на странице

Если нужно расположить на странице элементы, то используем группировку в `html.Div`
Например, если нужно заголовок сделать вверху, потом 2 блока рядом в линию и потом одни блок ниже, то
```
html.Div([
    html.Div(заголовок),
    html.Div([
        html.Div(блок 1),
        html.Div(блок 2)
    ], style={'display': 'flex', 'flexDirection': 'row'}),
    html.Div(блок 3)
])    
```

Захват нажатого бара на графике

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(id='click-data', children='output text', style={'Color': 'white'}), 
    dcc.Graph(id='graph', figure={
        'data': [
            go.Bar(x=['A', 'B', 'C'], y=[10, 20, 30])
        ]
    }),
])

@app.callback(
    Output('click-data', 'children'),
    [Input('graph', 'clickData')]
)
def display_click_data(click_data):
    if click_data:
        bar = click_data['points'][0]
        return f'Вы кликнули на бар {bar["x"]} с значением {bar["y"]}'
    else:
        return 'Нет данных о клике'

if __name__ == '__main__':
    app.run_server()

Update Graphs on Hover

https://dash.plotly.com/interactive-graphing

Если нам нужно, чтобы когда мышкой на одном графике наводим на предмет и появляется hover data, чтобы    
другие объекты обновлялись, например, чтобы на других графиках появлялась инфа только по выбраному элементу

In [ ]:
# Полная версия дашборда   
# https://dash.plotly.com/interactive-graphing

from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
# выбираем поля для осей  
# на скэттере будет это x и y
# а на правых графиках это будут y оси для одного и для другого
xaxis_column_name = 'Life expectancy at birth, total (years)'
yaxis_column_name = 'GDP growth (annual %)'
year_value = 1992
def make_scatter(xaxis_column_name, yaxis_column_name, year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear')

    fig.update_yaxes(title=yaxis_column_name, type='linear')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig

app.layout = html.Div([
    html.Div([
        dcc.Graph(
            figure = make_scatter(xaxis_column_name, yaxis_column_name, year_value),
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

])


def create_time_series(dff, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'))
def update_x_timeseries(hoverData):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'))
def update_y_timeseries(hoverData):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
!pip install dash_ag_grid

AgGrid   
Таблица с сортировкой

In [ ]:
import dash_ag_grid as dag
from dash import Dash, html
import pandas as pd

app = Dash()

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/ag-grid/space-mission-data.csv")

app.layout = html.Div([
    dag.AgGrid(
    rowData=df.to_dict("records"),
    columnDefs=[{"field": i} for i in df.columns]
    )
])

app.run(debug=True)

In [ ]:
dash_table.DataTable(
        id='datatable',
        columns=[
            {"name": i, "id": i, "deletable": True} if i == 'Category'
            else  {"name": i, "id": i, "deletable": True, 'type': 'numeric'} for i in df1.columns
        ],
        data=df1.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="multi",
        row_deletable=True,
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
        # style_data={ # параметры аналогчиные style_header
        #     'textAlign': 'left'
        # },
        style_header={
            'backgroundColor': 'white'
            , 'color': 'black' # Цвет текста
            # , 'fontSize': ''
            # , 'fontFamily': ''
            # , 'fontWeight': '' # normal or bold
            # , 'textAlign': 'left'
            # , 'borderWidth': ''
            # , 'border': 'solid' # Can be "solid", "dashed", "dotted", or "none".
            # , 'borderWidth': ''
            # , 'borderColor': ''
            # , 'padding': '10px'
        },
        style_data_conditional=[      
            # {
            #     'if': {
            #         'column_editable': True
            #     },
            #     'backgroundColor': 'Yellow',
            #     'color': 'white'
            # },                                       
            # {
            #     'if': {
            #         'column_id': 'Category'
            #     },
            #     'backgroundColor': 'blue',
            #     'color': 'white'
            # },                  
            {
                'if': {
                    'row_index': 'odd'
                },
                'backgroundColor': '#F7F7F7'
                , 'color': 'black'
            },
            # {
            #     'if': {
            #         'column_type': 'numeric'
            #     },
            #     'backgroundColor': 'rgba(228, 128, 128, 1)'
            # },            
            {
                'if': {
                    'row_index': 'even'
                },
                'backgroundColor': 'white'
                , 'color': 'black'
            },
            {
                'if': {
                    'state': 'selected'
                },
                'backgroundColor': 'white',  # Цвет для выбранной ячейки
                'borderColor': 'rgba(128, 60, 170, 0.9)', 
                'color': 'black'  # Цвет текста для выбранной ячейки
            },    
            {
                'if': {
                    'column_id': 'Value',
                    'filter_query': '{Value} > 20'
                },
                'backgroundColor': 'green',
                'color': 'white'
            },                    
        ]        
    ),  

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

colors = {
    'background': 'white',
    'text': 'black'
}

def generate_table(dataframe, max_rows=10, width = '100px', text_align = 'right'):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col, style={'width': width, 'text-align': text_align}) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col], style={'width': width, 'text-align': text_align}) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])
# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")
markdown_text = '''
### Dash and Markdown

Dash apps can be written in Markdown.
'''

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(children=[
    html.H1(
        children='Hello Dash',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    html.Div(children=[
            html.Div(children=[
            html.Label('Dropdown'),
            dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),

            html.Br(),
            html.Label('Multi-Select Dropdown'),
            dcc.Dropdown(['New York City', 'Montréal', 'San Francisco'],
                        ['Montréal', 'San Francisco'],
                        multi=True),

            html.Br(),
            html.Label('Radio Items'),
            dcc.RadioItems(['New York City', 'Montréal', 'San Francisco'], 'Montréal'),
        ], style={'padding': 10, 'flex': 1}),

        html.Div(children=[
            html.Label('Checkboxes'),
            dcc.Checklist(['New York City', 'Montréal', 'San Francisco'],
                        ['Montréal', 'San Francisco']
            ),

            html.Br(),
            html.Label('Text Input'),
            dcc.Input(value='MTL', type='text'),

            html.Br(),
            html.Label('Slider'),
            dcc.Slider(
                min=0,
                max=9,
                marks={i: f'Label {i}' if i == 1 else str(i) for i in range(1, 6)},
                value=5,
            ),
        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flexDirection': 'row'})
    # html.Div(children='Dash: A web application framework for your data.', style={
    #     'textAlign': 'center',
    #     'color': colors['text']
    # }),
    # dcc.Markdown(children=markdown_text),
    # dcc.Graph(
    #     id='example-graph-2',
    #     figure=fig
    # ),
    
    # # generate_table(df), 
    # dash_table.DataTable(
    #     id='datatable-interactivity',
    #     columns=[
    #         {"name": i, "id": i, "deletable": True} for i in df.columns
    #     ],
    #     data=df.to_dict('records'),
    #     editable=True,
    #     filter_action="native",
    #     sort_action="native",
    #     sort_mode="multi",
    #     row_selectable="multi",
    #     row_deletable=True,
    #     selected_rows=[],
    #     page_action="native",
    #     page_current= 0,
    #     page_size= 10,
    # )
], style={'backgroundColor': colors['background']})

if __name__ == '__main__':
    app.run(debug=True)

Callback

Если у нас график или таблица будут обновляться в callback функции,  
то в layout просто создаем блок с id и все, наполнение будет идти в callback функции

Dash App With Multiple Inputs

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px

import pandas as pd

app = Dash(__name__)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='xaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='xaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='yaxis-type',
                inline=True
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},

    )
])


@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    Input('year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name,
                     type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name,
                     type='linear' if yaxis_type == 'Linear' else 'log')

    return fig


if __name__ == '__main__':
    app.run(debug=True

Dash App With Multiple Outputs

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['x', html.Sup(2)]), html.Td(id='square')]),
        html.Tr([html.Td(['x', html.Sup(3)]), html.Td(id='cube')]),
        html.Tr([html.Td([2, html.Sup('x')]), html.Td(id='twos')]),
        html.Tr([html.Td([3, html.Sup('x')]), html.Td(id='threes')]),
        html.Tr([html.Td(['x', html.Sup('x')]), html.Td(id='x^x')]),
    ]),
])


@callback(
    Output('square', 'children'),
    Output('cube', 'children'),
    Output('twos', 'children'),
    Output('threes', 'children'),
    Output('x^x', 'children'),
    Input('num-multi', 'value'))
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x


if __name__ == '__main__':
    app.run(debug=True)

Dash App With Chained Callbacks  
You can also chain outputs and inputs together: the output of one callback function could be the input of another callback function.

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

all_options = {
    'America': ['New York City', 'San Francisco', 'Cincinnati'],
    'Canada': ['Montréal', 'Toronto', 'Ottawa']
}
app.layout = html.Div([
    dcc.RadioItems(
        list(all_options.keys()),
        'America',
        id='countries-radio',
    ),

    html.Hr(),

    dcc.RadioItems(id='cities-radio'),

    html.Hr(),

    html.Div(id='display-selected-values')
])


@callback(
    Output('cities-radio', 'options'),
    Input('countries-radio', 'value'))
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in all_options[selected_country]]


@callback(
    Output('cities-radio', 'value'),
    Input('cities-radio', 'options'))
def set_cities_value(available_options):
    return available_options[0]['value']


@callback(
    Output('display-selected-values', 'children'),
    Input('countries-radio', 'value'),
    Input('cities-radio', 'value'))
def set_display_children(selected_country, selected_city):
    return f'{selected_city} is a city in {selected_country}'


if __name__ == '__main__':
    app.run(debug=True)

Если нужно, чтобы обновление дашборда было по кнопке,  
например у нас поле ввода и нам нужно чтобы изменение было, когда человек полностью закончил ввод и готов к обновлению,  
то испольуем `State()` вместо `Input()`, а `Input()` ставим для нажатия кнопки, тогда колбэк сработает только после нажания кнопки
Тогда любыые изменения будут сохраняться в `State()`, но автоматически изменяться дашборд не будет,  
пока не сработает `Input()`,  
Note that we're triggering the callback by listening to the n_clicks property of the html.Button component.   
n_clicks is a property that gets incremented every time the component has been clicked on.  
It's available in every component in Dash HTML Components (dash.html), but most useful with buttons.

In [ ]:
from dash import Dash, dcc, html, Input, Output, State, callback

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Input(id='input-1-state', type='text', value='Montréal'),
    dcc.Input(id='input-2-state', type='text', value='Canada'),
    html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
    html.Div(id='output-state')
])


@callback(Output('output-state', 'children'),
              Input('submit-button-state', 'n_clicks'),
              State('input-1-state', 'value'),
              State('input-2-state', 'value'))
def update_output(n_clicks, input1, input2):
    return f'''
        The Button has been pressed {n_clicks} times,
        Input 1 is "{input1}",
        and Input 2 is "{input2}"
    '''


if __name__ == '__main__':
    app.run(debug=True)

You can also provide components directly as inputs and outputs without adding or referencing an id.  
Dash autogenerates IDs for these components.

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

app = Dash(__name__)

my_input = dcc.Input(value='initial value', type='text')
my_output = html.Div()

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        my_input
    ]),
    html.Br(),
    my_output
])


@callback(
    Output(my_output, 'children'),
    Input(my_input, 'value')
)
def update_output_div(input_value):
    return f'Output: {input_value}'


if __name__ == '__main__':
    app.run(debug=True)

In [5]:
import pandas as pd
pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv').head()

,country,pop,continent,lifeExp,gdpPercap
0,Afghanistan,31889923.0,Asia,43.828,974.580338
1,Albania,3600523.0,Europe,76.423,5937.029526
2,Algeria,33333216.0,Africa,72.301,6223.367465
3,Angola,12420476.0,Africa,42.731,4797.231267
4,Argentina,40301927.0,Americas,75.320,12779.379640


In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

# Incorporate data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

# Initialize the app
app = Dash()

table_data = [['Team', 'Wins', 'Losses', 'Ties'],
              ['Montréal<br>Canadiens', 18, 4, 0],
              ['Dallas Stars', 18, 5, 0],
              ['NY Rangers', 16, 5, 0],
              ['Boston<br>Bruins', 13, 8, 0],
              ['Chicago<br>Blackhawks', 13, 8, 0],
              ['LA Kings', 13, 8, 0],
              ['Ottawa<br>Senators', 12, 5, 0]]

fig = ff.create_table(table_data, height_constant=60)

# App layout
app.layout =  html.Div([
    html.Div(children='My First App with Data, Graph, and Controls',
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    html.Hr(),
    dcc.RadioItems(options=['pop', 'lifeExp', 'gdpPercap'], value='pop', id='controls-and-radio-item'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6),
    dcc.Graph(figure=fig, id='my_table'),
    dcc.Graph(figure={}, id='controls-and-graph')
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    return fig

@callback(
    Output(component_id='my_table', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    if col_chosen == 'pop':
        fig = ff.create_table(table_data, height_constant=60)
    if col_chosen == 'lifeExp':
        fig = ff.create_table(table_data[:2], height_constant=60)
    if col_chosen == 'gdpPercap':
        fig = ff.create_table(table_data[:5], height_constant=60)
                
    return fig
# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import dash
from dash.dependencies import Input, Output
from dash import dash_table
from dash import dcc
from dash import html
import pandas as pd
from dash.dash_table.Format import Group

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='field-dropdown',
        options=[{'label': continent, 'value': continent} for continent in df['continent'].unique()],
        multi=True,
        value='Asia',
        clearable=False
    ),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="multi",
        row_deletable=True,
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Div(id='datatable-interactivity-container')
])

@app.callback(Output('datatable-interactivity', 'data'), [Input('field-dropdown', 'value')])
def update_table(user_selection):
    """
    For user selections, return the relevant table
    """
    if isinstance(user_selection,str):
        df_to_ret = df[df['continent'] == user_selection]
    else:
        df_to_ret = df[df['continent'].isin(user_selection)]
    return df_to_ret.to_dict('records')

@app.callback(
    Output('datatable-interactivity-container', "children"),
    [Input('datatable-interactivity', "derived_virtual_data"),
     Input('datatable-interactivity', "derived_virtual_selected_rows")])
def update_graphs(rows, derived_virtual_selected_rows):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncracy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    if derived_virtual_selected_rows is None:
        derived_virtual_selected_rows = []

    dff = df if rows is None else pd.DataFrame(rows)

    colors = ['#7FDBFF' if i in derived_virtual_selected_rows else '#0074D9'
              for i in range(len(dff))]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [
                    {
                        "x": dff["country"],
                        "y": dff[column],
                        "type": "bar",
                        "marker": {"color": colors},
                    }
                ],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {
                        "automargin": True,
                        "title": {"text": column}
                    },
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        for column in ["pop", "lifeExp", "gdpPercap"] if column in dff
    ]


if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import datetime

import dash
from dash import dcc
from dash import html
import plotly
from dash.dependencies import Input, Output

# pip install pyorbital
from pyorbital.orbital import Orbital
satellite = Orbital('TERRA')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(
    html.Div([
        html.H4('TERRA Satellite Live Feed'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='live-update-graph'),
# этот элемент отвечает за обновление дэшборда
# в данном случае будет обновление раз в 1000 milliseconds        
        dcc.Interval(
            id='interval-component',
            interval=1*1000, # in milliseconds
            n_intervals=0
        )
    ])
)


@app.callback(Output('live-update-text', 'children'),
              [Input('interval-component', 'n_intervals')])
def update_metrics(n):
    lon, lat, alt = satellite.get_lonlatalt(datetime.datetime.now())
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.Span('Longitude: {0:.2f}'.format(lon), style=style),
        html.Span('Latitude: {0:.2f}'.format(lat), style=style),
        html.Span('Altitude: {0:0.2f}'.format(alt), style=style)
    ]


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              [Input('interval-component', 'n_intervals')])
def update_graph_live(n):
    satellite = Orbital('TERRA')
    data = {
        'time': [],
        'Latitude': [],
        'Longitude': [],
        'Altitude': []
    }

    # Collect some data
    for i in range(180):
        time = datetime.datetime.now() - datetime.timedelta(seconds=i*20)
        lon, lat, alt = satellite.get_lonlatalt(
            time
        )
        data['Longitude'].append(lon)
        data['Latitude'].append(lat)
        data['Altitude'].append(alt)
        data['time'].append(time)

    # Create the graph with subplots
    fig = plotly.tools.make_subplots(rows=2, cols=1, vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    fig.append_trace({
        'x': data['time'],
        'y': data['Altitude'],
        'name': 'Altitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 1, 1)
    fig.append_trace({
        'x': data['Longitude'],
        'y': data['Latitude'],
        'text': data['time'],
        'name': 'Longitude vs Latitude',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 2, 1)

    return fig

server = app.server

if __name__ == '__main__':
    server.run(
        ssl_context=('cert.pem', 'key.pem'),
        host='0.0.0.0',
        port='8080'
        )


In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.datasets import load_iris, load_wine

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


############ Loading Datasets ##################
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df["FlowerType"] = [iris.target_names[target] for target in iris.target]

wine = load_wine()

wine_df = pd.DataFrame(data=wine.data, columns=wine.feature_names)
wine_df["WineType"] = [wine.target_names[target] for target in wine.target]


apple_df = pd.read_csv("https://raw.githubusercontent.com/matplotlib/sample_data/master/aapl.csv")


################# Line Chart ##############################
chart1 = go.Figure()

chart1.add_trace(go.Scatter(x=apple_df.Date, y=apple_df.Open,
                            marker={"color":"tomato"},
                            mode="lines"))


chart1.update_layout(height=500,
                     xaxis_title="Date",
                     yaxis_title="Price ($)",
                     title="Apple Stock Prices [Apr-2019-Mar-2020]")

################# Scatter Plot ################################
chart2 = px.scatter(data_frame=wine_df,
                   x=wine.feature_names[0],
                   y=wine.feature_names[1],
                   color="WineType",
                   title="alcohol vs malic_acid color-encoded by wine type",
                   height=500,
                   )


################## Bar Chart ###############################                   
iris_avg_by_flower_type = iris_df.groupby(by="FlowerType").mean().reset_index()

chart3 = px.bar(data_frame=iris_avg_by_flower_type,
               x="FlowerType",
               y=iris.feature_names[0],
               title="Avg %s Per Flower Type"%iris.feature_names[0],
               height=500,
               )

#################### Creating App Object ############################               
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

####################### Setting Graphs as HTML Children ##############             
graph1 = dcc.Graph(
        id='graph1',
        figure=chart1,
        #className="eight columns"
    )

graph2 = dcc.Graph(
        id='graph2',
        figure=chart2,
        #className="five columns"
    )

graph3 = dcc.Graph(
        id='graph3',
        figure=chart3,
        #className="five columns"
    )


############### Creating Widgets For Each Graph #########################    
multi_select_line_chart = dcc.Dropdown(
        id="multi_select_line_chart",
        options=[{"value":label, "label":label} for label in ["Open", "Low", "High", "Close"]],
        value=["Open"],
        multi=True,
        clearable = False
    )

dropdown1_scatter_chart = dcc.Dropdown(
        id="dropdown1_scatter_chart",
        options=[{"value":label, "label":label} for label in wine.feature_names],
        value=wine.feature_names[0],
        className="six columns",
        clearable = False
    )

dropdown2_scatter_chart = dcc.Dropdown(
        id="dropdown2_scatter_chart",
        options=[{"value":label, "label":label} for label in wine.feature_names],
        value=wine.feature_names[1],
        className="six columns",
        clearable = False
    )

dropdown_bar_chart = dcc.Dropdown(
        id="dropdown_bar_chart",
        options=[{"value":label, "label":label} for label in iris.feature_names],
        value=iris.feature_names[0],
        clearable = False
    )


######################### Laying out Charts & Widgets to Create App Layout ##########
header = html.H2(children="Simple Dashboard With Widgets")

row1 = html.Div(children=[multi_select_line_chart, graph1], className="eight columns")

scatter_div = html.Div(children=[html.Div(children=[dropdown1_scatter_chart, dropdown2_scatter_chart], className="row") , graph2], className="six columns")

bar_div = html.Div(children=[dropdown_bar_chart, graph3], className="six columns")

row2 = html.Div(children=[scatter_div, bar_div], className="eight columns")

layout = html.Div(children=[header, row1, row2], style={"text-align": "center", "justifyContent":"center"})

############### Setting App Layout ########################################
app.layout = layout


################## Creating Callbacks for Each Widget ############################
@app.callback(Output('graph1', 'figure'), [Input('multi_select_line_chart', 'value')])
def update_line(price_options):
    chart1 = go.Figure()

    for price_op in price_options:
        chart1.add_trace(go.Scatter(x=apple_df.Date, y=apple_df[price_op],
                                mode="lines", name=price_op))


    chart1.update_layout(
                         xaxis_title="Date",
                         yaxis_title="Price ($)",
                         title="Apple Stock Prices [Apr-2019-Mar-2020]",
                         height=500,)
    return chart1



@app.callback(Output('graph2', 'figure'), [Input('dropdown1_scatter_chart', 'value'), Input('dropdown2_scatter_chart', 'value')])
def update_scatter(drop1, drop2):
    chart2 = px.scatter(data_frame=wine_df,
                   x=drop1,
                   y=drop2,
                   color="WineType",
                   title="%s vs %s color-encoded by wine type"%(drop1, drop2),
                   height=500,
                   )

    return chart2


@app.callback(Output('graph3', 'figure'), [Input('dropdown_bar_chart', 'value')])
def update_bar(bar_drop):
    chart3 = px.bar(data_frame=iris_avg_by_flower_type,
               x="FlowerType",
               y=bar_drop,
               title="Avg %s Per Flower Type"%bar_drop,
               height=500,
               )
    return chart3


################## Running App #####################################

if __name__ == "__main__":
    app.run_server(debug=True)